In [1]:
import pandas as pd
from pandas import DataFrame
import numpy as np
from datetime import datetime
import datetime as dt
import time

#Read the data - SAP, SFDC & Ecomm
SAP = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\01_SAP_Raw Report_NAC.xlsx")

SFDC = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\SFDC_Raw Nac Report.csv", parse_dates=['Date/Time Opened'], dayfirst=True, encoding = "cp1252")

#Remove duplicates on both data
SAP = SAP.drop_duplicates(subset=['Sales Document'])
SFDC = SFDC.drop_duplicates(subset=['SAP Order No.'])

#Remove false time from Document Date, Delivery Date & Goods Issue Date
SAP['Goods Issue Date'] = SAP['Goods Issue Date'].dt.strftime('%d/%m/%Y')
SAP['Document Date'] = SAP['Document Date'].dt.strftime('%d/%m/%Y')
SAP['Delivery Date'] = SAP['Delivery Date'].dt.strftime('%d/%m/%Y')

#Adjust SFDC date/time
SFDC['Date/Time Opened'] = (SFDC['Date/Time Opened']).dt.strftime('%Y-%m-%d %H:%M:%S')


In [2]:
#Rename some columns
SFDC.rename(columns = {'Case Number' : 'Case_Number', 'Date/Time Opened' : 'Rcvd_SFDC', 
                       'Number of subscribers activated' : 'Number_of_Subs_activated', 
                       'Number of subscribers renewed' : 'Number_of_Subs_Renewed'}, inplace = True)
SAP.rename(columns = {'Sales Office' : 'Order_Type', 'Goods Issue Date' : 'Goods_Date'}, inplace = True)


In [3]:
#Only show Sales Office from 9502

values = [9502]

SAP = SAP.loc[SAP['Order_Type'].isin(values)]

In [4]:
#Merge both data

SFDC['SAP Order No.']=SFDC['SAP Order No.'].apply(pd.to_numeric, errors='coerce').fillna(0).astype(float).dropna()
R1 = pd.merge(left=SAP, right=SFDC, left_on='Sales Document', right_on='SAP Order No.', how='left')


In [5]:
#Read the agents' list

agent = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\Agent list.xlsx")


In [6]:
#Merge agent list data on R1

R2 = pd.merge(left=R1, right=agent, left_on='User ID', right_on='ID', how='left')


In [7]:
#Remove non NAC orders

R2.dropna(subset=['Comment'], inplace=True)


In [8]:
#Define all the necessary status

s2 = pd.read_csv("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\SFDC_Raw Nac Report.csv", parse_dates=['Edit Date'], dayfirst=True, encoding = "cp1252")

#Adjust date/time
s2['Edit Date'] = (s2['Edit Date']).dt.strftime('%Y-%m-%d %H:%M:%S')

s2['ACT_Status_Date'] = s2['Edit Date']
s2['CARAT Date'] = s2['Edit Date']
s2['CAD Date'] = s2['Edit Date']
s2['CARAT'] = s2['Case Number']
s2['VALIDATION'] = s2['Case Number']
s2['PREORDER'] = s2['Case Number']
s2['BACKORDER'] = s2['Case Number']
s2['CAD'] = s2['Case Number']


In [9]:
#Check for Activation

act = s2[(s2['Old Value'] == 'Activation') | (s2['New Value'] == 'Activation')]

act = act[['Case Number', 'ACT_Status_Date']]

act = act.drop_duplicates(subset=['Case Number'])

In [10]:
#Check for Backorder

bo = s2[(s2['Old Value'] == 'Backorder') | (s2['New Value'] == 'Backorder') | (s2['Old Value'] == 'Allocation') |
       (s2['New Value'] == 'Allocation')]

bo = bo[['Case Number', 'BACKORDER']]

bo = bo.drop_duplicates(subset=['Case Number'])


In [11]:
#Check for CARAT

car = s2[(s2['Old Value'] == 'CARAT') | (s2['New Value'] == 'CARAT')]

car = car[['Case Number', 'CARAT Date', 'CARAT']]

car = car.drop_duplicates(subset=['Case Number'])

In [12]:
#Check for Validation

val = s2[(s2['Old Value'] == 'Validation') | (s2['New Value'] == 'Validation')]

val = val[['Case Number', 'VALIDATION']]

val = val.drop_duplicates(subset=['Case Number'])

In [13]:
#Check for Preorder

pre = s2[(s2['Old Value'] == 'Preorder') | (s2['New Value'] == 'Preorder')]

pre = pre[['Case Number', 'PREORDER']]

pre = pre.drop_duplicates(subset=['Case Number'])

In [14]:
#Check for CAD

cad = s2[(s2['Old Value'] == 'CAD') | (s2['New Value'] == 'CAD')]

cad = cad[['Case Number', 'CAD Date']]

cad = cad.drop_duplicates(subset=['Case Number'])

In [15]:
#Merge all status

from functools import reduce

data_frame = [act, car, bo, val, pre, cad]
status = reduce(lambda left, right: pd.merge(left, right, on=['Case Number'], how='outer'), data_frame)


In [16]:
f = status[['CARAT', 'BACKORDER', 'VALIDATION', 'PREORDER']]

status['ALL_EXCL'] = [ x[pd.notna(x)].tolist()[0] if len(x[pd.notna(x)].tolist()) > 0 else "" for x in f.values]


In [17]:
#Add the various status to table

R3 = pd.merge(left=R2, right=status, left_on='Case_Number', right_on='Case Number', how='left')


In [18]:
#Read SAP again so as to concatenate date & time

sp = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\01_SAP_Raw Report_NAC.xlsx")
sp['Created_in_SAP'] = pd.to_datetime(sp['Document Date']) + pd.to_timedelta(sp['Document Time'].astype(str))
sp['Dropped_to_WH'] = pd.to_datetime(sp['Delivery Date']) + pd.to_timedelta(sp['Delivery Time'].astype(str))
sp['Goods_Issue'] = pd.to_datetime(sp['Goods Issue Date']) + pd.to_timedelta(sp['Goods Issue Time'].astype(str))
sp.rename(columns = {'Sales Document' : 'Sales Doc'}, inplace = True)
sp_new = sp[['Sales Doc', 'Created_in_SAP', 'Dropped_to_WH', 'Goods_Issue']]

#Remove duplicate on sales document
sp_new = sp_new.drop_duplicates(subset=['Sales Doc'])


In [19]:
#Add SFDC recieved time to the sp table by joining sp to R3 using sale document column

R4 = pd.merge(left=R3, right=sp_new, left_on='Sales Document', right_on='Sales Doc', how='left')


In [1]:
#Calculate date without weekends and holidays

COL1 = "SFDC_Goods_Mins"
COL2 = "SFDC_SAP_Mins"
COL3 = "SAP_WH_Mins"
COL4 = "WH_Shipping_Mins"
COL5 = "SAP_Goods_Mins"


hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
R4[COL1] = 0
R4[COL2] = 0
R4[COL3] = 0
R4[COL4] = 0
R4[COL5] = 0


for index, row in R4.iterrows():
    try:
        sfdc_date = datetime.strptime(str(row["Rcvd_SFDC"]), "%Y-%m-%d %H:%M:%S")
        doc_time_date = datetime.strptime(str(row["Created_in_SAP"]), "%Y-%m-%d %H:%M:%S")
        del_time_date = datetime.strptime(str(row["Dropped_to_WH"]), "%Y-%m-%d %H:%M:%S")
        gd_time_date = datetime.strptime(str(row["Goods_Issue"]), "%Y-%m-%d %H:%M:%S")
                   
        R4.at[index, COL1] = get_diff(sfdc_date, gd_time_date)
        R4.at[index, COL2] = ifor_val = get_diff(sfdc_date, doc_time_date)
        R4.at[index, COL3] = get_diff(doc_time_date, del_time_date)
        R4.at[index, COL4] = get_diff(del_time_date, gd_time_date)
        R4.at[index, COL5] = get_diff(doc_time_date, gd_time_date)
    except Exception as e:
        #print(e)
        


IndentationError: expected an indented block (Temp/ipykernel_16992/1511858184.py, line 58)

In [2]:
COL6 = "WH_ACT_Mins"
COL7 = "ACT_Shipping_Mins"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0


SECONDS_IN_A_DAY = 86400
R4[COL6] = 0
R4[COL7] = 0

for index, row in R4.iterrows():
    try:
        act_time_date = datetime.strptime(str(row["ACT_Status_Date"]), "%Y-%m-%d %H:%M:%S")
        del_time_date = datetime.strptime(str(row["Dropped_to_WH"]), "%Y-%m-%d %H:%M:%S")
        gd_time_date = datetime.strptime(str(row["Goods_Issue"]), "%Y-%m-%d %H:%M:%S")
       
        
        R4.at[index, COL6] = get_diff(del_time_date, act_time_date)
        R4.at[index, COL7] = get_diff(act_time_date, gd_time_date)
       
    except Exception as e:
        #print(e)

IndentationError: expected an indented block (Temp/ipykernel_16992/1603559001.py, line 45)

In [3]:

COL8 = "CARAT_Processtime_Mins"

hd = pd.read_excel("C:\\WSS-NAC-Reports\\Automation Folder\\IQ Report\\Holidays.xlsx")
HOLIDAYS = hd['HolidayDate']

def get_diff(date1, date2):
    try:
        diff = abs((date2 - date1).total_seconds())
        days_between = (date2.date() - date1.date()).days
        weekdays_between = abs(np.busday_count(date2.date(), date1.date()))
        weekends_between = days_between - weekdays_between

        if date2.date() != date1.date():
            diff = diff - (weekends_between * SECONDS_IN_A_DAY)
            
        for holiday in HOLIDAYS:
            holiday_date = datetime.strptime(str(holiday), "%Y-%m-%d %H:%M:%S")
            if date1.date() <= holiday_date.date() <= date2.date():
                diff = diff -  SECONDS_IN_A_DAY


        return int(diff / 60)

    except Exception as e:
        print(e)
        return 0

SECONDS_IN_A_DAY = 86400
R4[COL8] = 0

for index, row in R4.iterrows():
    try:
        
        car_time_date = datetime.strptime(str(row["CARAT Date"]), "%Y-%m-%d %H:%M:%S")
        cad_time_date = datetime.strptime(str(row["CAD Date"]), "%Y-%m-%d %H:%M:%S")
        
        
        R4.at[index, COL8] = get_diff(car_time_date, cad_time_date)
    except Exception as e:
        #print(e)

IndentationError: expected an indented block (Temp/ipykernel_16992/4189694298.py, line 40)

In [23]:
#no_of_lines

R4['no_of_Lines'] = R4['Number_of_Subs_activated'] + R4['Number_of_Subs_Renewed']


In [24]:
#Export BBAN Result

NAC = R4[['Sales Document', 'Case_Number', 'Name of sold-to party', 'User ID', 'Agent_Name',  'Rcvd_SFDC','Created_in_SAP', 
         'Dropped_to_WH', 'ACT_Status_Date', 'Goods_Issue', 'Goods_Date', 'SFDC_Goods_Mins', 'SFDC_SAP_Mins', 'SAP_WH_Mins', 
          'WH_ACT_Mins', 'ACT_Shipping_Mins', 'WH_Shipping_Mins', 'SAP_Goods_Mins', 'Number_of_Subs_activated', 
          'Number_of_Subs_Renewed', 'CARAT_Processtime_Mins', 'Order_Type', 'Sales Group', 'BACKORDER', 'VALIDATION', 
          'PREORDER', 'CARAT', 'ALL_EXCL', 'no_of_Lines']]
export_excell = NAC.to_excel(r'C:\\WSS-NAC-Reports\\Automation Folder\\NAC Report\\BBAN_Output.xlsx', index=None, header=True)